## TOMTOM API :

In [1]:
import requests
import folium

In [ ]:
API_KEY = "XXXXX80YSuR0XRKrPNqJf4W13jq2P4lJk" #ici pour créer une clé API https://developer.tomtom.com/traffic-api/documentation/tomtom-maps/product-information/introduction

In [6]:
# Coordonnées 
start = (4.85, 45.75)
end = (4.83, 45.77)

# 1. Routing API : calculer l'itinéraire
route_url = (
    f"https://api.tomtom.com/routing/1/calculateRoute/"
    f"{start[1]},{start[0]}:{end[1]},{end[0]}/json"
    f"?traffic=true&key={API_KEY}"
)
route_data = requests.get(route_url).json()
points = route_data["routes"][0]["legs"][0]["points"]
summary = route_data["routes"][0]["summary"]

# Durée avec Routing API
print(f"Distance : {summary['lengthInMeters']/1000:.1f} km")
print(f"Durée avec trafic : {summary['travelTimeInSeconds']/60:.1f} min")

# 2. Flow API : récupérer trafic sur quelques points
def get_flow(point):
    url = (
        f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
        f"?point={point['latitude']},{point['longitude']}&key={API_KEY}"
    )
    return requests.get(url).json()["flowSegmentData"]

flow_start = get_flow(points[0])
flow_mid   = get_flow(points[len(points)//2])
flow_end   = get_flow(points[-1])

# Trafic Flow API
print("\n🚦 Données Flow API (point de départ)")
print(f"- Vitesse actuelle : {flow_start['currentSpeed']} km/h")
print(f"- Vitesse libre : {flow_start['freeFlowSpeed']} km/h")
print(f"- Niveau de congestion : {flow_start['confidence']} (confiance)")

# 3. Folium : tracer la carte
m = folium.Map(location=[45.76, 4.83], zoom_start=13)

# Polyline du trajet
coords = [(p["latitude"], p["longitude"]) for p in points]
folium.PolyLine(coords, color="blue", weight=4).add_to(m)

# Marqueurs avec trafic
folium.Marker(
    coords[0],
    popup=f"Départ Lyon\nVitesse actuelle: {flow_start['currentSpeed']} km/h"
).add_to(m)

folium.Marker(
    coords[len(coords)//2],
    popup=f"Milieu\nVitesse actuelle: {flow_mid['currentSpeed']} km/h"
).add_to(m)

folium.Marker(
    coords[-1],
    popup=f"Arrivée Marseille\nVitesse actuelle: {flow_end['currentSpeed']} km/h"
).add_to(m)

# Afficher carte
m

Distance : 4.4 km
Durée avec trafic : 23.5 min

🚦 Données Flow API (point de départ)
- Vitesse actuelle : 22 km/h
- Vitesse libre : 22 km/h
- Niveau de congestion : 1 (confiance)


### Caching avec sqlite
Pour préserver le nombre de requêtes à l'API

In [7]:
import sqlite3, time, json

# Connexion à la base SQLite
conn = sqlite3.connect("cache.db")
c = conn.cursor()
c.execute("""CREATE TABLE IF NOT EXISTS cache (
    key TEXT PRIMARY KEY,
    response TEXT,
    timestamp REAL
)""")
conn.commit()

def get_cached_response(key, ttl=300):
    """Récupère une réponse du cache si elle est encore valide"""
    c.execute("SELECT response, timestamp FROM cache WHERE key=?", (key,))
    row = c.fetchone()
    if row:
        response, ts = row
        if time.time() - ts < ttl:
            return json.loads(response)
    return None

def set_cache(key, response):
    """Stocke une réponse dans le cache"""
    c.execute("REPLACE INTO cache (key, response, timestamp) VALUES (?, ?, ?)",
              (key, json.dumps(response), time.time()))
    conn.commit()

def get_route(start, end):
    key = f"route_{start}_{end}"
    cached = get_cached_response(key, ttl=86400)  # 24h pour la géométrie
    if cached:
        return cached
    
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{start[1]},{start[0]}:{end[1]},{end[0]}/json?traffic=true&key={API_KEY}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    set_cache(key, data)
    return data

def get_flow(point):
    key = f"flow_{point}"
    cached = get_cached_response(key, ttl=300)  # 5 min pour le trafic
    if cached:
        return cached
    
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?point={point[1]},{point[0]}&key={API_KEY}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    set_cache(key, data)
    return data

# Exemple d'utilisation
start = (4.8357, 45.7640)   # Lyon
end   = (5.3698, 43.2965)   # Marseille

route = get_route(start, end)
flow  = get_flow(start)

print("Durée avec trafic :", route["routes"][0]["summary"]["travelTimeInSeconds"]/60, "min")
print("Vitesse actuelle au départ :", flow["flowSegmentData"]["currentSpeed"], "km/h")

Durée avec trafic : 176.28333333333333 min
Vitesse actuelle au départ : 16 km/h
